In [ ]:
import csv
import json
from collections import Counter
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import folium
from folium.plugins import HeatMap
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from haversine import haversine
from scipy.spatial import cKDTree



# Définir le chemin du fichier CSV
CHEMIN_FICHIER_CSV = "trees.csv"

def charger_donnees(fichier_csv):

    try:
        with open(fichier_csv, mode='r', encoding='utf-8-sig') as fichier:
            lecteur = csv.reader(fichier)
            en_tete = next(lecteur)
            donnees = list(lecteur)
        return en_tete, donnees
    except FileNotFoundError:
        print("Erreur : Fichier introuvable.")
        exit()

# Ecemple d'utlisatio de loc et iloc pour afficher les données
def test_loc_iloc():
    df = pd.read_csv('trees.csv')
    print("Test loc et iloc :")
    print(df.loc[0:5, 'GENRE_BOTA':'ANNEEDEPLANTATION'])
    print(df.iloc[0:5, 1:4])

def afficher_en_tete(donnees, en_tete):

    print("\nEn-tête du fichier :", en_tete)
    nombre_lignes = int(input("Combien de lignes afficher ? "))
    print("Premières lignes :", donnees[:nombre_lignes])

def convertir_en_dataframe(donnees, en_tete):
    """Convertit les données en DataFrame pandas."""
    return pd.DataFrame(donnees, columns=en_tete)

def analyser_annees_plantation(df):
   
    if "ANNEEDEPLANTATION" in df.columns:
        df['ANNEEDEPLANTATION'] = pd.to_numeric(df['ANNEEDEPLANTATION'], errors='coerce').dropna().astype(int)
        print(f"\nNombre total d'arbres recensés : {len(df)}")
        
        # Afficher des années spécifiques
        nombre_annees = int(input("Combien d'années afficher ? "))
        print("Années de plantation :", df['ANNEEDEPLANTATION'].unique()[:nombre_annees])
        
        # Statistiques
        print(f"Année la plus ancienne : {df['ANNEEDEPLANTATION'].min()}")
        print(f"Année la plus récente : {df['ANNEEDEPLANTATION'].max()}")
        print(f"Moyenne d'arbres plantés par an : {len(df) / df['ANNEEDEPLANTATION'].nunique():.2f}")
    else:
        print("Erreur : La colonne 'ANNEEDEPLANTATION' est introuvable.")

def analyse_genre_botanique(df):
    
    if "GENRE_BOTA" in df.columns:
        genres = df["GENRE_BOTA"].dropna()
        top_genre = genres.value_counts().idxmax()
        proportion_top = genres.value_counts(normalize=True).max() * 100
        print(f"\nLe genre botanique le plus représenté est {top_genre} ({proportion_top:.2f}%).")
        print("Proportions des genres botaniques :")
        print(genres.value_counts(normalize=True) * 100)
    else:
        print("Erreur : La colonne 'GENRE_BOTA' est introuvable.")


def verifier_loi_metro(df):
    
    if "GENRE_BOTA" in df.columns and "ESPECE" in df.columns:
        total_arbres = len(df)
        proportion_genre = sum(nb_arbres / total_arbres for nb_arbres in df["GENRE_BOTA"].value_counts() if nb_arbres / total_arbres > 0.2)
        proportion_espece = sum(nb_arbres / total_arbres for nb_arbres in df["ESPECE"].value_counts() if nb_arbres / total_arbres > 0.1)
        print("\nRespect de la loi métro :")
        print(f"Proportion de genre botanique > 20% : {proportion_genre:.2%}")
        print(f"Proportion d'espèce botanique > 10% : {proportion_espece:.2%}")
    else:
        print("Erreur : Colonnes 'GENRE_BOTA' ou 'ESPECE' manquantes.")

def tracer_histogramme(df):

    if "ANNEEDEPLANTATION" in df.columns:
        plt.figure(figsize=(12, 6))
        df["ANNEEDEPLANTATION"].value_counts().sort_index().plot(kind='bar')
        plt.xlabel("Année de plantation")
        plt.ylabel("Nombre d'arbres")
        plt.title("Nombre d'arbres plantés par année")
        plt.show()
    else:
        print("Erreur : La colonne 'ANNEEDEPLANTATION' est introuvable.")

def tracer_histogramme2(df):

    compteur_annees = Counter(df["ANNEEDEPLANTATION"].dropna().astype(int))
    if "ANNEEDEPLANTATION" in df.columns:
        if compteur_annees:
            annees = sorted(compteur_annees.keys())
            arbres = [compteur_annees[annee] for annee in annees]
            proportions = [arbres[i] / sum(arbres[:i + 1]) * 100 for i in range(len(arbres))]
            plt.figure(figsize=(30, 10))
            plt.plot(annees, proportions)
            plt.xlabel("Année de plantation")
            plt.ylabel("Proportion d'arbres plantés (%)")
            plt.title("Évolution de la diversification des plantations")
            plt.show()

# Afficher la représentation géographique des arbres de Grenoble en utilisant la colonne "GeoJSON" (seulement 1000 arbres aleatoires)
def representation_geographique2(df):
    
    if "GeoJSON" not in df.columns:
        print("Erreur : La colonne 'GeoJSON' est introuvable.")
        return

    # Créer une carte centrée sur Grenoble
    carte = folium.Map(location=[45.1885, 5.7245], zoom_start=13)

    # Sélectionner un échantillon aléatoire de 1000 arbres
    arbres = df.dropna(subset=["GeoJSON"]).sample(n=1000, random_state=1)

    for _, arbre in arbres.iterrows():
        try:
            # Convertir la chaîne JSON en dictionnaire Python
            geo_data = json.loads(arbre["GeoJSON"])
            
            # Extraire les coordonnées (long, lat) -> (lat, long)
            longitude, latitude = geo_data["coordinates"]
            
            # Ajouter un marqueur sur la carte
            folium.Marker(
                [latitude, longitude],
                popup=f"{arbre.get('GENRE_BOTA', 'Inconnu')} {arbre.get('ESPECE', '')}"
            ).add_to(carte)

        except (json.JSONDecodeError, KeyError, TypeError):
            print(f"Erreur lors du traitement des coordonnées pour l'arbre: {arbre}")

    # Afficher la carte directement dans VS Code Notebook
    display(carte)

# Creation et ecriture dans un nouveau fichier csv sans les arbres manquant de données
def creer_fichier_csv_sans_arbres_manquants(df):
                          
    if "GENRE_BOTA" in df.columns and "ESPECE" in df.columns and "ANNEEDEPLANTATION" in df.columns and "GeoJSON" in df.columns:
        # Filtrer les arbres manquants de données
        arbres_manquants = df[df.isnull().any(axis=1)]
        arbres_complets = df.dropna()
        print(f"\nNombre d'arbres manquants : {len(arbres_manquants)}")
        print(f"Nombre d'arbres complets : {len(arbres_complets)}")
        
        # Créer un nouveau fichier CSV
        fichier_csv_sans_manquants = "arbres_sans_manquants.csv"
        arbres_complets.to_csv(fichier_csv_sans_manquants, index=False)
        print(f"Fichier '{fichier_csv_sans_manquants}' créé avec succès.")
    else:
        print
        ("Erreur : Colonnes 'GENRE_BOTA', 'ESPECE', 'ANNEEDEPLANTATION' ou 'GeoJSON' manquantes.")   

# Creation et ecrtirue dans un nouveau fichier csv avec les arbres qui manquent de données seulement
def creer_fichier_csv_arbres_manquants(df):
    
    if "GENRE_BOTA" in df.columns and "ESPECE" in df.columns and "ANNEEDEPLANTATION" in df.columns and "GeoJSON" in df.columns:
        # Filtrer les arbres manquants de données
        arbres_manquants = df[df.isnull().any(axis=1)]
        arbres_complets = df.dropna()
        print(f"\nNombre d'arbres manquants : {len(arbres_manquants)}")
        print(f"Nombre d'arbres complets : {len(arbres_complets)}")
        
        # Créer un nouveau fichier CSV
        fichier_csv_manquants = "arbres_manquants.csv"
        arbres_manquants.to_csv(fichier_csv_manquants, index=False)
        print(f"Fichier '{fichier_csv_manquants}' créé avec succès.")
    else:
        print("Erreur : Colonnes 'GENRE_BOTA', 'ESPECE', 'ANNEEDEPLANTATION' ou 'GeoJSON' manquantes.")

def creer_fichier_csv_sans_arbres_manquants(df):
    
    colonnes_requises = ["GENRE_BOTA", "ESPECE", "ANNEEDEPLANTATION", "GeoJSON"]

    # Vérifier la présence des colonnes requises
    if not all(col in df.columns for col in colonnes_requises):
        print("Erreur : Colonnes 'GENRE_BOTA', 'ESPECE', 'ANNEEDEPLANTATION' ou 'GeoJSON' manquantes.")
        return

    # Filtrer les arbres complets (sans données manquantes)
    arbres_complets = df.dropna(subset=colonnes_requises).reset_index(drop=True)

    # Affichage des statistiques
    print(f"\nNombre d'arbres complets : {len(arbres_complets)}")
    
    # Créer un nouveau fichier CSV
    fichier_csv_sans_manquants = "arbres_sans_manquants.csv"
    arbres_complets.to_csv(fichier_csv_sans_manquants, index=False)
    print(f"Fichier '{fichier_csv_sans_manquants}' créé avec succès.")

def creer_fichier_csv_arbres_manquants(df):

    colonnes_requises = ["GENRE_BOTA", "ESPECE", "ANNEEDEPLANTATION", "GeoJSON"]

    # Vérifier la présence des colonnes requises
    if not all(col in df.columns for col in colonnes_requises):
        print("Erreur : Colonnes 'GENRE_BOTA', 'ESPECE', 'ANNEEDEPLANTATION' ou 'GeoJSON' manquantes.")
        return

    # Filtrer les arbres manquants de données
    arbres_manquants = df[df.isnull().any(axis=1)].reset_index(drop=True)

    # Affichage des statistiques
    print(f"\nNombre d'arbres manquants : {len(arbres_manquants)}")
    
    # Créer un nouveau fichier CSV
    fichier_csv_manquants = "arbres_manquants.csv"
    arbres_manquants.to_csv(fichier_csv_manquants, index=False)
    print(f"Fichier '{fichier_csv_manquants}' créé avec succès.")

# Calcul des 10 arbres les plus proches de Vif (45.1094, 5.6656)
def arbres_proches(df):
    if "GeoJSON" not in df.columns:
        print("Erreur : La colonne 'GeoJSON' est introuvable.")
        return

    # Coordonnées de Vif
    vif = (45.1094, 5.6656)

    # Calcul de la distance entre Vif et chaque arbre
    df["Distance"] = df["GeoJSON"].apply(lambda geojson: haversine(json.loads(geojson)["coordinates"], vif))

    # Affichage des 10 arbres les plus proches
    arbres_proches = df.nsmallest(10, "Distance")
    print("\nLes 10 arbres les plus proches de Vif :")
    print(arbres_proches[["GENRE_BOTA", "ESPECE", "Distance"]])

# Calcul de l'arbre le plus solitaire
def arbre_le_plus_solitaire(df):
    
    if "GeoJSON" not in df.columns:
        print("Erreur : La colonne 'GeoJSON' est introuvable.")
        return

    # Extraire les coordonnées géographiques
    arbres_coords = []
    arbres_infos = []  # Pour stocker les infos à afficher dans le popup
    for _, arbre in df.dropna(subset=["GeoJSON"]).iterrows():
        try:
            geo_data = json.loads(arbre["GeoJSON"])
            longitude, latitude = geo_data["coordinates"]
            arbres_coords.append((latitude, longitude))  # Format (lat, long)
            arbres_infos.append(arbre)  # Stocker l'arbre complet
        except (json.JSONDecodeError, KeyError, TypeError):
            continue

    # Vérifier qu'il y a assez de points pour calculer les distances
    if len(arbres_coords) < 2:
        print("Erreur : Pas assez de points pour déterminer l'arbre le plus solitaire.")
        return

    # Utilisation de KDTree pour calculer les distances efficacement
    tree = cKDTree(arbres_coords)
    distances, _ = tree.query(arbres_coords, k=2)  # k=2 car le 1er point est lui-même

    # Trouver l'arbre le plus éloigné
    max_distance_index = np.argmax(distances[:, 1])
    arbre_solitaire = arbres_infos[max_distance_index]

    # Création de la carte centrée sur l'arbre le plus solitaire
    carte = folium.Map(location=arbres_coords[max_distance_index], zoom_start=13)

    # Marqueur rouge pour l'arbre le plus solitaire
    folium.Marker(
        location=arbres_coords[max_distance_index],
        popup=f"{arbre_solitaire.get('GENRE_BOTA', 'Inconnu')} {arbre_solitaire.get('ESPECE', 'Inconnu')}\nDistance max : {distances[max_distance_index, 1]:.2f} km",
        icon=folium.Icon(color='red')
    ).add_to(carte)

    # Affichage de la carte
    display(carte)

# Calcul de l'arbre le plus entouré et laffichage sur une nouvelle carte
def arbre_le_plus_entoure(df):
    
    if "GeoJSON" not in df.columns:
        print("Erreur : La colonne 'GeoJSON' est introuvable.")
        return

    # Extraire les coordonnées géographiques
    arbres_coords = []
    arbres_infos = []  # Pour stocker les infos à afficher dans le popup
    for _, arbre in df.dropna(subset=["GeoJSON"]).iterrows():
        try:
            geo_data = json.loads(arbre["GeoJSON"])
            longitude, latitude = geo_data["coordinates"]
            arbres_coords.append((latitude, longitude))  # Format (lat, long)
            arbres_infos.append(arbre)  # Stocker l'arbre complet
        except (json.JSONDecodeError, KeyError, TypeError):
            continue

    # Vérifier qu'il y a assez de points pour calculer les distances
    if len(arbres_coords) < 2:
        print("Erreur : Pas assez de points pour déterminer l'arbre le plus entouré.")
        return

    # Utilisation de KDTree pour calculer les distances efficacement
    tree = cKDTree(arbres_coords)
    distances, _ = tree.query(arbres_coords, k=2)  # k=2 car le 1er point est lui-même

    # Trouver l'arbre le plus entouré
    min_distance_index = np.argmin(distances[:, 1])
    arbre_entoure = arbres_infos[min_distance_index]

    # Création de la carte centrée sur l'arbre le plus entouré
    carte = folium.Map(location=arbres_coords[min_distance_index], zoom_start=13)

    # Marqueur vert pour l'arbre le plus entouré
    folium.Marker(
        location=arbres_coords[min_distance_index],
        popup=f"{arbre_entoure.get('GENRE_BOTA', 'Inconnu')} {arbre_entoure.get('ESPECE', 'Inconnu')}\nDistance min : {distances[min_distance_index, 1]:.2f} km",
        icon=folium.Icon(color='green')
    ).add_to(carte)


    # Affichage de la carte
    display(carte)


def test_groupby(df):
    if "ANNEEDEPLANTATION" in df.columns:
        print("\nNombre d'arbres plantés par année :")
        print(df.groupby("ANNEEDEPLANTATION").size())
    else:
        print("Erreur : La colonne 'ANNEEDEPLANTATION' est introuvable;")

def main():
    """Programme principal."""
    en_tete, donnees = charger_donnees(CHEMIN_FICHIER_CSV)
    df = convertir_en_dataframe(donnees, en_tete)
    # test_groupby(df)
    # test_loc_iloc()
    # afficher_en_tete(donnees, en_tete)
    # analyser_annees_plantation(df)
    # analyse_genre_botanique(df)
    # verifier_loi_metro(df)
    # arbres_proches(df)
    # creer_fichier_csv_sans_arbres_manquants(df)
    # creer_fichier_csv_arbres_manquants(df)
    # distance_moyenne_entre_arbres(df)
    # tracer_histogramme(df)
    # tracer_histogramme2(df)   
    # representaion_geographique(df)
    # representation_geographique2(df)
    # arbre_le_plus_solitaire(df)
    # arbre_le_plus_entoure(df)
    analyse_genre_botanique(df)

if __name__ == "__main__":
    main()
        
# Non ordonné, unique, mutable (sauf frozenset)


Le genre botanique le plus représenté est Acer (527600.00%).
Proportions des genres botaniques :
GENRE_BOTA
Acer           527600
Platanus       466300
Pinus          211800
Tilia          177000
Fraxinus       150600
                ...  
Castanea          100
Nyssa             100
Sciadopitys       100
Sorbopyrus        100
Poncinos          100
Name: count, Length: 113, dtype: int64


In [ ]:

# loc[] : accès par étiquette
# iloc[] : accès par position
import csv
import json
from collections import Counter
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import folium
from folium.plugins import HeatMap
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from haversine import haversine
from scipy.spatial import cKDTree


df = pd.read_csv('trees.csv')
vif = (45.1094, 5.6656)

# exemple d'utilisation de drop et commentaire a chaque ligne pour expliquer lutilisaion de drop
# df = df.dropna(subset=["GeoJSON"]) # Supprimer les lignes avec des valeurs manquantes dans la colonne "GeoJSON"
# print(df)
# df = df.dropna(subset=["GENRE_BOTA", "ESPECE", "ANNEEDEPLANTATION", "GeoJSON"]) # Supprimer les lignes avec des valeurs manquantes dans les colonnes spécifiées
# print(df)

# exemple d'utilisation de drop tout court et commentaire a chaque ligne pour expliquer lutilisaion de drop
df = df.drop(df[df["GENRE_BOTA"].isnull()].index) # Supprimer les lignes avec des valeurs manquantes dans la colonne "GENRE_BOTA"
print(df)

# exemple d'utilisation de apply et commentaire a chaque ligne pour expliquer lutilisaion de apply
df["Distance"] = df["GeoJSON"].apply(lambda geojson: haversine(json.loads(geojson)["coordinates"], vif)) # apply utile pour appliquer une fonctions sur une collonnes (un peu comme for)
print(df)

# Exemple d'utilisan de .head et commentaire pour expliquer son utilisation
print(df.head()) # Afficher les premières lignes du DataFrame
print(df.head(10)) # Afficher les 10 premières lignes du DataFrame

# Exemple d'utilisation de value_counts et commentaire pour expliquer son utilisation
print(df["ANNEEDEPLANTATION"].value_counts()) # Compter les valeurs uniques dans la colonne "ANNEEDEPLANTATION"
print(df["ANNEEDEPLANTATION"].value_counts(normalize=True)) # Compter les valeurs uniques dans la colonne "ANNEEDEPLANTATION" et afficher les proportions

# Exemple d'utilisation de sort_values et commentaire pour expliquer son utilisation
print(df["ANNEEDEPLANTATION"].value_counts().sort_values()) # Trier les valeurs uniques par ordre croissant
print(df["ANNEEDEPLANTATION"].value_counts().sort_values(ascending=False)) # Trier les valeurs uniques par ordre décroissant

# exemple d'utilisation de groupby et commentaire pour expliquer son utilisation
print(df.groupby("ANNEEDEPLANTATION").size()) # Grouper les données par année de plantation et compter le nombre d'arbres par année




SyntaxError: Missing parentheses in call to 'print'. Did you mean print(...)? (1657299350.py, line 32)